## **DATA ANALYSIS - PH Twitter Mis/Disinformamtion on Redtagging**

In [4]:
# Import the necessary libraries
from typing import ItemsView
import tweepy, sys, re
import pandas as pd
import sched, time, datetime
import csv, openpyxl, xlsxwriter
import pprint

Read and store the contents of the dataset

In [6]:
# Open the dataset workbook
workbook = openpyxl.load_workbook(filename = 'Dataset - Group 32.xlsx')
worksheet = workbook['Data']
worksheet.max_column

# Convert to a Pandas dataframe
data = worksheet.values
df = pd.DataFrame(data, columns=next(data)[0:])
df



,ID,Timestamp,Tweet URL,Group,Collector,Category,Topic,Keywords,Account handle,Account name,...,Content type,Replies,Retweets,Quote Tweets,Views,Rating,Reasoning,Remarks,Reviewer,Review
0,"=Text(D2,""00"")&""-""&ROW(D2)-1",21/02/23 16:19:00,https://twitter.com/amylauderdake/status/13303...,32.0,"Aloveros, Aira Mae",REDT,Red tagging students from different universities,UP NPA PUP,@amylauderdake,amylauderdake,...,Rational,0.0,0.0,0.0,None,UNPROVEN,Accuses that the said universities are NPA bre...,Location and bio are not specified,None,None
1,"=Text(D3,""00"")&""-""&ROW(D3)-1",21/02/23 16:20:10,https://twitter.com/SamukaNimoUy/status/152575...,32.0,"Aloveros, Aira Mae",REDT,Red tagging students from UP and PUP,UP NPA PUP,@SamukaNimoUy,-Nessa-,...,Rational,0.0,0.0,0.0,None,UNPROVEN,Accuses that the said universities are NPA bre...,Location is not specified,None,None
2,"=Text(D4,""00"")&""-""&ROW(D4)-1",21/02/23 16:25:35,https://twitter.com/Shaider_de/status/13519219...,32.0,"Delos Reyes, Rey",REDT,Red tagging students from UP and PUP,UP NPA PUP,@Shaider_de,Dai Sawamura,...,Rational,0.0,0.0,0.0,None,UNPROVEN,Accuses that leaders of NPA comes from PUP and...,None,None,None
3,"=Text(D5,""00"")&""-""&ROW(D5)-1",21/02/23 17:23:35,https://twitter.com/n4qpu/status/1351844058935...,32.0,"Doros, Joshua Lloyd",REDT,Red tagging students from universities,TERORISTA UP NPA,@n4qpu,浜崎 夏海 🏅,...,Emotional,0.0,0.0,0.0,None,MISLEADING,Suggests that student activists will recruit t...,None,None,None
4,"=Text(D6,""00"")&""-""&ROW(D6)-1",21/02/23 17:25:35,https://twitter.com/JethroGamez/status/1293145...,32.0,"Doros, Joshua Lloyd",REDT,Red tagging students from UP,TERORISTA UP NPA,@JethroGamez,jethjeth ♥️💚,...,Emotional,0.0,0.0,0.0,None,UNPROVEN,Accuses UP of being controlled by the CPP-NPA-...,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,"=Text(D165,""00"")&""-""&ROW(D165)-1",24/03/23 18:43:24,https://twitter.com/iamsparky79/status/1571659...,32.0,"Delos Reyes, Rey",REDT,Red tagging students from state universities,(UPD OR PUP OR DLSU OR Ateneo) AND (Komunista ...,@iamsparky79,sparky,...,Rational,0.0,2.0,0.0,None,UNPROVEN,The allegation regarding UP being infiltrated ...,Location and bio are not specified,None,None
164,"=Text(D166,""00"")&""-""&ROW(D166)-1",24/03/23 18:43:24,https://twitter.com/xOrigin24x/status/15595863...,32.0,"Delos Reyes, Rey",REDT,Red tagging students from state universities,(UPD OR PUP OR DLSU OR Ateneo) AND (Komunista ...,@xOrigin24x,xOriginx,...,Emotional,0.0,1.0,0.0,None,MISLEADING,Specified that activists are automatically NPA...,Location is not specified,None,None
165,"=Text(D167,""00"")&""-""&ROW(D167)-1",24/03/23 18:43:24,https://twitter.com/keizerinj/status/154533381...,32.0,"Delos Reyes, Rey",REDT,Red tagging students from state universities,(UPD OR PUP OR DLSU OR Ateneo) AND (Komunista ...,@keizerinj,Yondaime,...,Emotional,0.0,1.0,0.0,None,UNPROVEN.MISLEADING,Specified that universities are being thought ...,Location and bio are not specified,None,None
166,"=Text(D168,""00"")&""-""&ROW(D168)-1",24/03/23 18:43:24,https://twitter.com/keizerinj/status/154533085...,32.0,"Delos Reyes, Rey",REDT,Red tagging students from state universities,(UPD OR PUP OR DLSU OR Ateneo) AND (Komunista ...,@keizerinj,Yondaime,...,Rational,0.0,0.0,0.0,None,UNPROVEN.MISLEADING,Specified that universities are being thought ...,Location and bio are not specified,None,None
